<h1> High Quality Momentum Trading Strategy </h1>

Calculating the number of stocks to be bought for the top 20 High Quality Momentum stocks in the S&P 500 Index for a user specific portfolio.

In [22]:
#Importing libraries

import numpy as np 
import pandas as pd 
import requests
from scipy import stats
import math

#Importing the constituents of the S&P 500

stocks= pd.read_csv('constituents.csv')

#Viewing the constituents

stocks

,Ticker,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [23]:
# Acquiring the IEX API Token
from secrets import IEX_CLOUD_API_TOKEN

<h1> Making API Calls and Creating our dataframe</h1>

In [24]:
#We use batch API calls here to improve the performance of our code
#reference https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One Year Price Return', 'Number of Shares to Buy']


<h1>High Quality Momentum Trading strategy</h1>

To have an effective Moment Trading Strategy, we need to ensure that the stocks we plan to invest in are High Quality Momentum stocks. High Quality Momentum or HQM stocks are preffered over Low Quality Momentum stocks as they show a steady growth momentum over a larger period of time and are not triggered by rare and specific incidents.

In [25]:
%%capture
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Stocks', 
                'One Year Price Return', 
                'One Year Return Percentile',
                'Six Month Price Return',
                'Six Month Return Percentile',
                'Three Month Price Return',
                'Three Month Return Percentile',
                'One Month Price Return',
                'One Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
                            


<h1>Calculating the percentile scores</h1>
In this part, we will be calculating the momentum percentiles of the enlisted stocks over the periods of:
<ul>
<li>One year price return</li>
<li>Six month price return</li>
<li>Three month price return</li>
<li>One month price return</li>
</ul>

In [26]:
time_periods = [
                'One Year',
                'Six Month',
                'Three Month',
                'One Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
   
hqm_dataframe

,Ticker,Price,Number of Stocks,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,149.62,N/A,-0.234194,0.13,-0.176736,0.34,-0.150627,0.17,-0.043135,0.54,N/A
1,AOS,61.32,N/A,-0.063654,0.43,-0.013616,0.58,-0.171421,0.12,-0.062552,0.47,N/A
2,ABT,118.68,N/A,-0.02636,0.47,-0.079855,0.5,-0.028036,0.44,-0.015519,0.72,N/A
3,ABBV,157.09,N/A,0.495527,0.96,0.4912,0.99,0.173552,0.92,-0.015005,0.73,N/A
4,ABMD,299.24,N/A,-0.186989,0.23,-0.202015,0.27,-0.00055,0.53,-0.133515,0.23,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CE,146.91,N/A,-0.092104,0.39,-0.137153,0.38,-0.121041,0.25,-0.024153,0.66,N/A
96,CNC,81.72,N/A,0.331532,0.9,0.147222,0.81,0.06081,0.73,-0.058419,0.48,N/A
97,CNP,32.1,N/A,0.361589,0.93,0.215731,0.9,0.157452,0.88,0.031405,0.84,N/A
98,CDAY,59.89,N/A,-0.421277,0.02,-0.554615,0.01,-0.188898,0.08,-0.169986,0.13,N/A


<h1>Calculating the HQM Score</h1> 

The High Quality Momentum score is the arithmetic mean of the afforementioned four momentum percentiles.

In [27]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

# Selecting our top 20 HQM stocks

hqm_dataframe.sort_values(by = 'HQM Score', ascending= True)
hqm_dataframe = hqm_dataframe[:21]


<h1> Accepting the desired portfolio amount and issuing reccomendations</h1>

Here, we accept the user's amount to be invested and return the number of stocks the user should buy of each respective High Quality Momentum Stock.


In [28]:
# Asking for the value of portfolio

def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter Portfolio Value:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("Please enter a valid amount:")
        portfolio_size = input("Enter Portfolio Value:")

portfolio_input()
print(portfolio_size)

#Calculating the number of stocks

position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Stocks'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe




5000000


,Ticker,Price,Number of Stocks,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,149.62,1591,-0.234194,0.13,-0.176736,0.34,-0.150627,0.17,-0.043135,0.54,0.295
1,AOS,61.32,3882,-0.063654,0.43,-0.013616,0.58,-0.171421,0.12,-0.062552,0.47,0.4
2,ABT,118.68,2006,-0.02636,0.47,-0.079855,0.5,-0.028036,0.44,-0.015519,0.72,0.5325
3,ABBV,157.09,1515,0.495527,0.96,0.4912,0.99,0.173552,0.92,-0.015005,0.73,0.9
4,ABMD,299.24,795,-0.186989,0.23,-0.202015,0.27,-0.00055,0.53,-0.133515,0.23,0.315
5,ACN,326.7,728,0.056881,0.64,-0.145733,0.37,-0.089469,0.33,-0.073609,0.43,0.4425
6,ATVI,80.1,2972,-0.167201,0.25,-0.026749,0.54,-0.029996,0.43,-0.0396,0.56,0.445
7,ADM,95.86,2483,0.563193,0.98,0.476322,0.98,0.272663,1.0,0.00481,0.76,0.93
8,ADBE,422.02,564,-0.235275,0.12,-0.380297,0.06,-0.201043,0.07,-0.08095,0.4,0.1625
9,AAP,217.68,1093,0.168791,0.72,-0.012888,0.59,0.006559,0.57,0.043923,0.92,0.7
